<a href="https://colab.research.google.com/github/remzicam/chatbot_medium/blob/main/chatbot_medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install chainlit pyngrok langchain optimum auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ -q

In [2]:
!ngrok config add-authtoken <paste_your_token_here>

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
from pyngrok import ngrok
print(ngrok.connect(8000).public_url)

https://a82f-34-136-28-232.ngrok-free.app


In [4]:
%%writefile app.py
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
import chainlit as cl

model_name_or_path = "TheBloke/zephyr-7B-beta-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto")

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512
    )

llm=HuggingFacePipeline(pipeline=pipe)

template = """
<|system|>
You are a helpful assistant that provides information and engages in casual conversation.
Respond naturally to user queries and provide useful information.
Please, write a single reply only!
</s>
<|user|>
Current conversation:
{history}
Question: {input}
</s>
<|assistant|>
"""

prompt = PromptTemplate(input_variables=["history", "input"], template=template)
memory=ConversationBufferWindowMemory(k=3)


@cl.on_chat_start
async def start():
    llm_chain = ConversationChain(prompt=prompt, llm=llm, memory=memory)
    cl.user_session.set("llm_chain", llm_chain)


@cl.on_message
async def main(message:cl.message):
    llm_chain = cl.user_session.get("llm_chain")
    cb = cl.AsyncLangchainCallbackHandler( )
    cb.answer_reached = True
    res = await cl.make_async(llm_chain)(message.content, callbacks=[cb])
    await cl.Message(content=res['response']).send()


Writing app.py


In [ ]:
!chainlit run app.py

2023-11-26 10:45:34 - Created default config file at /content/.chainlit/config.toml
2023-11-26 10:45:43.981068: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-26 10:45:43.987862: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-26 10:45:43.987918: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-26 10:45:47.405008: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
tokenizer_config.json: 100% 1.43k/1.43k [00:00<00:00, 5.80MB/s]
tokenizer.model: 100% 493k/493k [00:00<00:00, 9.89MB/s]
tokenizer.json: 100% 1.80M/1.80M [